In [1]:
#import pandas to load the datasets into dataframes
import pandas as pd

# Importing the Seaborn library for advanced data visualization (as sns)
import seaborn as sns

# Importing the Matplotlib library for basic plotting functionalities (as plt)
import matplotlib.pyplot as plt

# Importing datetime for later convertions
from datetime import datetime

# Matplotlib inline to visualize Matplotlib graphs
%matplotlib inline

# Configuration to set so that all the Seaborn figures come out with this size
%config Inlinebackend.figure_format= 'retina'

# Set the Seaborn context to "poster" for larger text and figures
sns.set_context("poster")

# Set the default figure size for Seaborn plots
sns.set(rc={"figure.figsize": (12., 6.)})

# Set the Seaborn style to "whitegrid" for a white background with gridlines
sns.set_style("whitegrid")


In [ ]:
df = pd.read_csv("vehicle_sales_dataset.csv")

In [98]:
df['vin'].nunique()

465768

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 465768 entries, 0 to 472324
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   year                   465768 non-null  int64  
 1   brand                  465768 non-null  object 
 2   model                  465768 non-null  object 
 3   category               465768 non-null  object 
 4   transmission           465768 non-null  object 
 5   vin                    465768 non-null  object 
 6   state_of_registration  465768 non-null  object 
 7   condition              465768 non-null  float64
 8   odometer               465768 non-null  float64
 9   color                  465768 non-null  object 
 10  interior_color         465768 non-null  object 
 11  seller                 465768 non-null  object 
 12  sellingprice           465768 non-null  float64
 13  saledate               465768 non-null  object 
dtypes: float64(3), int64(1), object(10)
memor

In [21]:
# Let's clean the date column and keep only the day, month and year and put it in this format 'YYYY-MM-DD' for mySQL's delight (default format for later date arithmetic)

# Function to cut the bullshit part of the date to get only month, day and year.
def cut(date_str):
    # Extract the part 
    return str(date_str)[4:15]

# Apply the conversion function to the 'saledate' column
df['saledate'] = df['saledate'].apply(cut)

In [32]:
# Convert the month to a number and the whole date to match mySQL's default format
month_mapping = {
    'Jan': '01',
    'Feb': '02',
    'Mar': '03',
    'Apr': '04',
    'May': '05',
    'Jun': '06',
    'Jul': '07',
    'Aug': '08',
    'Sep': '09',
    'Oct': '10',
    'Nov': '11',
    'Dec': '12'
}

# Convert 'saledate' column to datetime
df['saledate'] = pd.to_datetime(df['saledate'].str[:3].map(month_mapping) + df['saledate'].str[3:], format='%m %d %Y')

# Format 'saledate' column to 'YYYY-MM-DD'
df['saledate'] = df['saledate'].dt.strftime('%Y-%m-%d')

In [ ]:
# now every date is converted to work with mySQL's default format

In [50]:
# Drop rows with missing values in at least one place
df = df.dropna()

In [67]:
df = df.reset_index(drop=True)

In [59]:
# droping every row that has a duplicate vin number so we can use the vin as a key in mySQL later
df = df.drop_duplicates(subset=['vin'])

In [ ]:
# Now we can work with rows where no data is missing

In [87]:
# Create a dictionary to map seller names to unique IDs
unique_sellers = df['seller'].unique()
seller_id_map = {seller: idx for idx, seller in enumerate(unique_sellers, start=1)}


In [88]:
# Add a new column 'seller_id' to the DataFrame
df['seller_id'] = df['seller'].map(seller_id_map)

In [92]:
# Let's export the cleaned data to a csv file to see in google sheets
df.to_excel('sellerid_python_cleaned_car_dataset.xlsx')

In [95]:
# Create a DataFrame from the seller_id_map dictionary
seller_df = pd.DataFrame(seller_id_map.items(), columns=['seller_id', 'seller_name'])

# Now seller_df contains two columns: 'seller_id' and 'seller_name'
# 'seller_id' contains unique IDs for each seller name
# 'seller_name' contains the corresponding seller names

# Export the DataFrame to Excel so we can use it as a table in mySQL
seller_df.to_excel('seller_mapping.xlsx', index=False)